<a href="https://colab.research.google.com/github/harshvs4/Coding_Drill_Down_MNIST/blob/main/Exp6_augmentation_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [17]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [18]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomRotation((-6.0, 6.0), fill=(1,)),                
                                       transforms.RandomAffine(degrees=7, shear=10, translate=(0.1, 0.1), scale=(0.8, 1.2)),
                                       transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.40, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [19]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [20]:
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=2, pin_memory=True) if use_cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

In [21]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 8, 3, padding = 0, bias = False),  # (1*26*26), (8*26*26), RF - 3*3
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.Dropout(0.05),

        nn.Conv2d(8, 16, 3, padding = 0, bias = False), #(8*24*24), (16*24*24), RF - 5*5
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(0.05)
    )

    #Transition block
    self.trans1 = nn.Sequential(
        nn.MaxPool2d(2, 2) , #(16*12*12), RF - 10*10
        nn.Conv2d(16, 8, 1, padding = 0, bias = False),  #(16*12*12), (8*12*12), RF - 10*10
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.Dropout(0.05)
    )

    self.conv2 = nn.Sequential(
        nn.Conv2d(8, 10, 3, padding = 0, bias = False), #(8*10*10), (10*10*10), RF - 12*12
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Dropout(0.05),

        nn.Conv2d(10, 16, 3, padding = 0, bias = False), #(10*8*8), (16*8*8), RF - 14*14
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(0.05),

        nn.Conv2d(16, 16, 3, padding = 0, bias = False), #(16*8*8),(16*8*8), RF - 14*14 
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout(0.05)
    )

    #Output block
    self.avgpool2d = nn.AvgPool2d(6,6) #RF - 28*28
    self.conv3 = nn.Conv2d(16, 10, 1, padding = 0, bias = False) #(16*6*6), (10*6*6), RF - 28*28
    

  def forward(self, x):
       x = self.conv1(x)
       x = self.trans1(x)
       x = self.conv2(x)
       x = self.avgpool2d(x)
       x = self.conv3(x)
       x = x.view(-1,10)
       return F.log_softmax(x,dim=1)

In [22]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
             ReLU-11            [-1, 8, 12, 12]               0
      BatchNorm2d-12            [-1, 8, 12, 12]              16

In [23]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()
    
    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [24]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 1


Loss=0.43878963589668274 Batch_id=468 Accuracy=76.40: 100%|██████████| 469/469 [00:39<00:00, 11.92it/s]



Test set: Average loss: 0.1050, Accuracy: 9732/10000 (97.32%)

EPOCH: 2


Loss=0.24539239704608917 Batch_id=468 Accuracy=93.89: 100%|██████████| 469/469 [00:35<00:00, 13.23it/s]



Test set: Average loss: 0.0631, Accuracy: 9804/10000 (98.04%)

EPOCH: 3


...


Test set: Average loss: 0.0214, Accuracy: 9931/10000 (99.31%)

EPOCH: 14


Loss=0.033007532358169556 Batch_id=468 Accuracy=97.67: 100%|██████████| 469/469 [00:34<00:00, 13.56it/s]



Test set: Average loss: 0.0256, Accuracy: 9910/10000 (99.10%)

EPOCH: 15


Loss=0.1254301220178604 Batch_id=468 Accuracy=97.64: 100%|██████████| 469/469 [00:34<00:00, 13.51it/s]



Test set: Average loss: 0.0210, Accuracy: 9939/10000 (99.39%)

